In [31]:
import numpy as np
import pandas as pd
from models import TSP, HopfieldNetwork

In [32]:
def read_instance(filepath):
    with open(f"{filepath}", "r") as f:
        content = f.readlines()
        content[0] = content[0].split(":")[-1].strip()[1:-1]
        tour = np.array([int(x) for x in content[0].split(",")])
        cost = float(content[1].split(":")[-1].strip())
        return (tour, cost)

In [44]:
instance = read_instance("test/tsp/instance2.sol")
print(instance[-1])

1039.77


In [45]:
tsp = TSP.from_instance_file("test/tsp/instance2.tsp", A=50000, B=50000, C=2000, D=1)

In [46]:
import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr
options = {}
with gp.Env(params=options) as env, gp.Model(env=env) as model:
    # Formulate problem
    problem = tsp
    # Define the interaction matrix (Q) and field vector (c)
    Q = problem.T[-2] # Interaction matrix
    c = problem.I # Field vector
    
    # Create a new Gurobi model
    model = Model("Qua")
    
    # Add binary variables
    n = Q.shape[0]
    var = [model.addVar(vtype=GRB.BINARY, name=f"x{i}") for i in range(n)]
    
    # Set up the quadratic objective
    objective = QuadExpr()
    
    # Add quadratic terms from Q
    for i in range(n):
        for j in range(n):
            if Q[i, j] != 0:
                objective.add(-0.5*Q[i, j] * var[i] * var[j])
    
    # Add linear terms from c
    for i in range(n):
        if c[i] != 0:
            objective.add(-c[i] * var[i])
    
            
    objective.addConstant(problem.offset)
    
    # Set the objective
    model.setObjective(objective, GRB.MINIMIZE)
    model.setParam("MIPFocus", 3)
    
    # Optimize the model
    model.optimize()
    
    
    gurobi_solution = []
    # Display results
    if model.status == GRB.OPTIMAL:
        print("Optimal value:", model.objVal)
        for v in var:
            print(f"{v.varName} = {v.x}")
            gurobi_solution.append(v.x)
    else:
        print("No optimal solution found.")
        for v in var:
            print(f"{v.varName} = {v.x}")
            gurobi_solution.append(v.x)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2595968
Academic license 2595968 - for non-commercial use only - registered to s0___@uni-bonn.de
Set parameter MIPFocus to value 3
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPFocus  3

Academic license 2595968 - for non-commercial use only - registered to s0___@uni-bonn.de
Optimize a model with 0 rows, 2500 columns and 0 nonzeros
Model fingerprint: 0x8ec9fd7e
Model has 3126250 quadratic objective terms
Variable types: 0 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+05, 2e+05]
  QObjective range [4e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 5000000.0000
Found heuristic solution: objective 5105.09469

x1942 = -0.0
x1943 = -0.0
x1944 = -0.0
x1945 = -0.0
x1946 = -0.0
x1947 = -0.0
x1948 = -0.0
x1949 = -0.0
x1950 = -0.0
x1951 = -0.0
x1952 = -0.0
x1953 = -0.0
x1954 = -0.0
x1955 = -0.0
x1956 = -0.0
x1957 = -0.0
x1958 = -0.0
x1959 = -0.0
x1960 = -0.0
x1961 = -0.0
x1962 = -0.0
x1963 = -0.0
x1964 = -0.0
x1965 = -0.0
x1966 = -0.0
x1967 = -0.0
x1968 = -0.0
x1969 = -0.0
x1970 = -0.0
x1971 = -0.0
x1972 = -0.0
x1973 = -0.0
x1974 = -0.0
x1975 = -0.0
x1976 = -0.0
x1977 = -0.0
x1978 = -0.0
x1979 = -0.0
x1980 = -0.0
x1981 = -0.0
x1982 = -0.0
x1983 = -0.0
x1984 = -0.0
x1985 = -0.0
x1986 = -0.0
x1987 = -0.0
x1988 = -0.0
x1989 = -0.0
x1990 = -0.0
x1991 = -0.0
x1992 = 1.0
x1993 = -0.0
x1994 = -0.0
x1995 = -0.0
x1996 = -0.0
x1997 = -0.0
x1998 = -0.0
x1999 = -0.0
x2000 = -0.0
x2001 = -0.0
x2002 = -0.0
x2003 = -0.0
x2004 = -0.0
x2005 = -0.0
x2006 = -0.0
x2007 = -0.0
x2008 = -0.0
x2009 = -0.0
x2010 = -0.0
x2011 = -0.0
x2012 = 1.0
x2013 = -0.0
x2014 = -0.0
x2015 = -0.0
x2016 = -0.0
x2017 = -0.0
x2018 = -0.0
x

In [47]:
from models import HopfieldNetwork
hopfield = HopfieldNetwork(tsp.T, tsp.I, tsp.offset)

print(len(tsp.interpret(np.array(gurobi_solution))))
tsp_sol = tsp.interpret(np.array(gurobi_solution))
print(tsp.interpret(np.array(gurobi_solution)))
print(tsp.get_cost(np.array(gurobi_solution)))
def is_legal(arr):
    print(np.unique(arr))
    return len(np.unique(arr)) == len(arr)

print(is_legal(tsp_sol))
print(np.array(gurobi_solution).reshape((tsp.n, tsp.n)))
print(hopfield.compute_energy(np.array(gurobi_solution), debug=True))

50
[22. 49. 15.  2. 23.  6. 20. 48. 19. 47. 44. 10. 40.  4.  0. 38. 45. 29.
 43. 16. 13. 31. 24. 41. 17. 27. 35. 36. 18. 12.  1.  3. 42. 33. 32.  5.
 26. 14.  7. 34. 21. 30. 39. 46. 11.  9. 25.  8. 37. 28.]
1926.0567363142902
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
True
[[-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 ...
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0.  1. -0. ... -0. -0. -0.]]
First term (inhibitory row connections): 0.0
First term (inhibitory column connections): 0.0
First term (Data term): 3852.11347262858
First term (global inhibitory connections): 10000000.0
Second term (Magnetic Field term): 10000000.0
Energy (with constant term): 1926.0567363146693
Constant term: 5000000
If V is a valid tour then the energy with the constant term is equa